In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

In [ ]:
cwd = os.path.join(os.getcwd())

dir_path = os.path.join(cwd, 'raw_datasets/era5')

In [ ]:
minyear = 1950
maxyear = 2023

lon_Barter_Island = -143.75
lat_Barter_Island = 70.0

lon_offshore = -143.75
lat_offshore = 70.25

In [ ]:
def mask_data_spatial(ds, lon, lat):
    
    spatial_mask = (ds.longitude == lon) * (ds.latitude == lat)

    return ds.where(spatial_mask, drop=True)

In [ ]:
locations = []


def convert_to_csv(var_name, minyear, maxyear, lon, lat, save_path=None, lat_offshore=70.25, lat_Barter_Island=70.0, save=False):
    
    time = []
    data = []

    years_with_data = np.arange(minyear, maxyear+1, 1)

    for year in years_with_data:
        
        fp = os.path.join(dir_path, var_name + "_" + str(year) + ".nc")
        
        ds = xr.open_dataset(fp)
                
        masked_ds = mask_data_spatial(ds, lon, lat).squeeze(drop=True)
        
        colname = list(ds.keys())[0]
        
        ds.close()

        time_year = list(masked_ds.time.values)
       
        print(time_year)
       
        time = time + time_year
        
        data += list(masked_ds[colname].values)
        
        masked_ds.close()
            
    df = pd.DataFrame({"time":time})

    if not save_path:
        save_path = var_name
    
    df[save_path] = data
    
    if lat == lat_offshore:
        locations.append((save_path, lon_offshore, lat_offshore, 'offshore'))
    else:
        locations.append((save_path, lon_Barter_Island, lat_Barter_Island, 'onshore'))
    
    if save:
        df.to_csv(os.path.join("ts_datasets/", save_path + ".csv"))
    
    print(f"succesful for variable {save_path}")
    
    return df

In [ ]:
# convert_to_csv('10m_u_component_of_wind', minyear, maxyear, lon_offshore, lat_offshore)

In [ ]:
# offshore_vars = [
#     "sea_ice_cover", 
#     "sea_surface_temperature", 
#     "10m_u_component_of_wind",
#     "10m_v_component_of_wind"
#     "soil_temperature_level_1_offs",
#     "soil_temperature_level_2_offs",
#     "soil_temperature_level_3_offs",
#     "soil_temperature_level_4_offs"
# ]

dfs = [
convert_to_csv('10m_u_component_of_wind', minyear, maxyear, lon_offshore, lat_offshore),
convert_to_csv('10m_v_component_of_wind', minyear, maxyear, lon_offshore, lat_offshore),
convert_to_csv('2m_temperature', minyear, maxyear, lon_Barter_Island, lat_Barter_Island),
convert_to_csv('mean_surface_latent_heat_flux', minyear, maxyear, lon_Barter_Island, lat_Barter_Island),
convert_to_csv('mean_surface_net_long_wave_radiation_flux', minyear, maxyear, lon_Barter_Island, lat_Barter_Island),
convert_to_csv('mean_surface_net_short_wave_radiation_flux', minyear, maxyear, lon_Barter_Island, lat_Barter_Island),
convert_to_csv('sea_ice_cover', minyear, maxyear, lon_offshore, lat_offshore),
convert_to_csv('sea_surface_temperature', minyear, maxyear, lon_offshore, lat_offshore),
convert_to_csv('snow_depth', minyear, maxyear, lon_Barter_Island, lat_Barter_Island),
convert_to_csv('soil_temperature_level_1', minyear, maxyear, lon_Barter_Island, lat_Barter_Island),
convert_to_csv('soil_temperature_level_2', minyear, maxyear, lon_Barter_Island, lat_Barter_Island),
convert_to_csv('soil_temperature_level_3', minyear, maxyear, lon_Barter_Island, lat_Barter_Island),
convert_to_csv('soil_temperature_level_4', minyear, maxyear, lon_Barter_Island, lat_Barter_Island),
convert_to_csv('soil_temperature_level_1', minyear, maxyear, lon_offshore, lat_offshore, save_path='soil_temperature_level_1_offs'),
convert_to_csv('soil_temperature_level_2', minyear, maxyear, lon_offshore, lat_offshore, save_path='soil_temperature_level_2_offs'),
convert_to_csv('soil_temperature_level_3', minyear, maxyear, lon_offshore, lat_offshore, save_path='soil_temperature_level_3_offs'),
convert_to_csv('soil_temperature_level_4', minyear, maxyear, lon_offshore, lat_offshore, save_path='soil_temperature_level_4_offs'),
]

In [ ]:
print(dfs[0])

In [ ]:
df_loc = pd.DataFrame(data=tslocations, columns=['var', 'lon', 'lat', 'note'])
df_loc.to_csv(os.path.join("ts_datasets/", "variable_locations.txt"))

In [ ]:
time = pd.read_csv(os.path.join("ts_datasets/", "2m_temperature.csv"))["time"].values

total_df = pd.DataFrame(time, columns=["time"])
total_df.head()

In [ ]:
dir_list = [
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "2m_temperature",
    "mean_surface_latent_heat_flux",
    "mean_surface_net_long_wave_radiation_flux",
    "mean_surface_net_short_wave_radiation_flux",
    "sea_ice_cover",
    "sea_surface_temperature",
    "snow_depth",
    "soil_temperature_level_1",
    "soil_temperature_level_2",
    "soil_temperature_level_3",
    "soil_temperature_level_4",
    "soil_temperature_level_1_offs",
    "soil_temperature_level_2_offs",
    "soil_temperature_level_3_offs",
    "soil_temperature_level_4_offs",
]

In [ ]:
for dir, df in zip(dir_list, dfs):
    total_df[dir] = df[dir].values
    
    

In [ ]:
# for dir in dir_list:
    
#     fpath = os.path.join("ts_datasets", dir + ".csv")
    
#     with open(fpath) as f:
        
#         array = pd.read_csv(f).iloc[:,2]
        
#         total_df[dir] = array

In [ ]:
total_df.to_csv(os.path.join("ts_datasets/", "era5.csv"))